In [3]:
import requests
import json
import os
import sqlalchemy
import pandas as pd
import mdapi_functions as md
from IPython.display import display, Markdown
import markdown
from bokeh.plotting import figure, show
from bokeh.models import (Span, TabPanel, Tabs, ColumnDataSource, DataCube,
                          GroupingInfo, StringFormatter, SumAggregator,
                          TableColumn, HoverTool)
from bokeh.io import output_notebook
from math import pi
from datetime import datetime
import datacite_api_functions as dcf
import numpy as np
import data_doc_helper as dh

In [4]:
ds = dh.NHSEDataSet("CANCER")
dataset = ds.dataset

In [5]:
dss1 = md.get_md_api_dss()
dss1["source_table"] = dss1["source"] + "_" + dss1["table"]
dsvs1 = md.get_md_api_dsvs()
dsvs1 = dsvs1[dsvs1["source"] == "nhsd"]
dsvs1["source"] = "NHSE"
dsvs1["version_num"] = dsvs1["version_num"].fillna("v0001")
def rm_aux_dss(x):
    if x.split("_")[0] in ["CSDS", "IAPT", "MHSDS", "HESOP", "HESAE"]:
        return x.split("_")[0].upper()
    else:
        return x.upper()

dsvs1["table"] = dsvs1["table"].apply(lambda x: rm_aux_dss(x))

def rm_aux_dss_full(x):
    if x.split("_")[0] in ["CSDS", "IAPT", "MHSDS", "HESOP", "HESAE"]:
        return x.split("_")[0] + "_" + x.split("_")[-1]
    else:
        return x

dsvs1["table_full"] = dsvs1["table_full"].apply(lambda x: rm_aux_dss_full(x))
dsvs1 = dsvs1.sort_values("num_columns", ascending=False)
dsvs1 = dsvs1.drop_duplicates(subset=["table_full"])
dsvs1["source_table"] = dsvs1["source"] + "_" + dsvs1["table"]
dsvs1["version_num"] = dsvs1["version_num"].\
    apply(lambda x: int(x.replace("v", "")))

def rename_reg_dss(table, vdate):
    return table + "_" + str(int(vdate))
dsvs1["table"] = dsvs1.apply(lambda row: rename_reg_dss(row["table"], row["version_date"]) if row["table"] in ["CANCER", "DEMOGRAPHICS", "MORTALITY"] else row["table"], axis=1)

def rename_reg_src_tbl(src, tbl):
    return src + "_" + tbl
dsvs1["source_table"] = dsvs1.apply(lambda row: rename_reg_src_tbl(row["source"], row["table"]) if row["source_table"] in ["NHSE_CANCER", "NHSE_DEMOGRAPHICS", "NHSE_MORTALITY"] else row["source_table"], axis=1)

def infill_vdates(vdate, vnum):
        vdict = {1: 20221221.0, 2: 20230413.0, 3: 20240426.0}
        if np.isnan(vdate):
            return vdict[vnum]
        else:
            return vdate

dsvs1["version_date"] = dsvs1.apply(lambda row: infill_vdates(row["version_date"], row["version_num"]), axis=1)
ds_dois = dcf.get_doi_datasets()
ds_dois = ds_dois[ds_dois["state"] == "findable"]
ds_dois["source_table"] = ds_dois["attributes.titles"].apply(lambda x: x[1]["title"])
ds_dois["attributes.version"] = ds_dois["attributes.version"].apply(lambda x: int(x))
dsvsf = dsvs1.merge(ds_dois, left_on=["source_table", "version_num"], right_on=["source_table", "attributes.version"])[["source_table", "version_num", "version_date", "num_participants", "num_columns", "num_rows", "id"]]
if dataset == "HESAPC":
    dsvs_i = dsvsf[dsvsf["source_table"] == "NHSE_" + dataset]
else:
    dsvs_i = dsvsf[dsvsf["source_table"].str.startswith("NHSE_" + dataset)]

dsvs_i = dsvs_i.sort_values("version_date")


dsvs_i["version_date"] = dsvs_i["version_date"].apply(lambda x: datetime.strftime(datetime.strptime(str(int(x)), "%Y%m%d"), "%d %b %Y"))
dsvs_i["num_participants"] = dsvs_i["num_participants"].apply(lambda x: "N/A" if np.isnan(x) else int(x))
dsvs_i["num_columns"] = dsvs_i["num_columns"].apply(lambda x: int(x))
dsvs_i["num_rows"] = dsvs_i["num_rows"].apply(lambda x: int(x))
dsvs_i["Change Log"] = dsvs_i["id"].apply(lambda x: md.make_hlink("https://api.test.datacite.org/dois/" + x + "/activities", x + "/activities"))
dsvs_i["id"] = dsvs_i["id"].apply(lambda x: md.make_hlink("https://doi.org/" + x, x))

if dataset in ["CANCER", "MORTALITY", "DEMOGRAPHICS"]:
    dsvs_i["version_num"] = dsvs_i.apply(lambda row: str(row["version_num"]) + " (" + row["version_date"] + ")", axis=1)

dsvs_i = dsvs_i.rename(columns = {"source_table": "Name in TRE", "version_num": "Version Number", "version_date": "Version Date", "num_participants": "Participant Count", "num_columns": "Number of Variables", "num_rows": "Number of Observations", "id": "DOI"}).set_index("Version Number")
dsvs_i_T = dsvs_i.T.reset_index().rename(columns={"index": "Version"})

dh.DocHelper.style_table("_", dsvs_i_T)

Version,2 (06 Jan 2022),1 (06 Jan 2022),1 (02 Mar 2022),2 (02 Mar 2022),1 (15 Jul 2022),1 (16 Jul 2022),1 (10 Nov 2022),1 (28 Dec 2022),2 (28 Dec 2022),2 (02 Mar 2023),2 (24 Apr 2024),2 (23 May 2024)
Name in TRE,NHSE_CANCER_20220106,NHSE_CANCER_20220106,NHSE_CANCER_20220302,NHSE_CANCER_20220302,NHSE_CANCER_20220715,NHSE_CANCER_20220716,NHSE_CANCER_20221110,NHSE_CANCER_20221228,NHSE_CANCER_20221228,NHSE_CANCER_20230302,NHSE_CANCER_20240424,NHSE_CANCER_20240523
Version Date,06 Jan 2022,06 Jan 2022,02 Mar 2022,02 Mar 2022,15 Jul 2022,16 Jul 2022,10 Nov 2022,28 Dec 2022,28 Dec 2022,02 Mar 2023,24 Apr 2024,23 May 2024
Participant Count,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
Number of Variables,22,21,21,21,21,21,21,21,21,21,21,21
Number of Observations,22520,22520,22728,22728,2245,24973,3507,27579,27579,27579,33682,34039
DOI,10.83126/ukllc-dataset-00062-02,10.83126/ukllc-dataset-00062-01,10.83126/ukllc-dataset-00061-01,10.83126/ukllc-dataset-00061-02,10.83126/ukllc-dataset-00063-01,10.83126/ukllc-dataset-00060-01,10.83126/ukllc-dataset-00059-01,10.83126/ukllc-dataset-00058-01,10.83126/ukllc-dataset-00058-02,10.83126/ukllc-dataset-00064-02,10.83126/ukllc-dataset-00066-02,10.83126/ukllc-dataset-00065-02
Change Log,10.83126/ukllc-dataset-00062-02/activities,10.83126/ukllc-dataset-00062-01/activities,10.83126/ukllc-dataset-00061-01/activities,10.83126/ukllc-dataset-00061-02/activities,10.83126/ukllc-dataset-00063-01/activities,10.83126/ukllc-dataset-00060-01/activities,10.83126/ukllc-dataset-00059-01/activities,10.83126/ukllc-dataset-00058-01/activities,10.83126/ukllc-dataset-00058-02/activities,10.83126/ukllc-dataset-00064-02/activities,10.83126/ukllc-dataset-00066-02/activities,10.83126/ukllc-dataset-00065-02/activities


In [6]:
def ds_doi(ss, ds: str):
    """Returns UKLLC DOI of LPS dataset

    Args:
        x (str): dataset e.g. "IAPT" or "HESCC"

    Returns:
        str: UKLLC DOI of dataset or "DOI TBC" if none minted
    """

    doi_ds = dcf.get_doi_datasets()[dcf.get_doi_datasets()["state"] == "findable"]
    doi_ds["source_table"] = doi_ds["attributes.titles"].apply(lambda x: x[1]["title"] if len(x) > 1 else "NA")

    doi_ds = doi_ds[doi_ds["source_table"] == ss + "_" + ds]
    doi_ds = doi_ds.sort_values(by="attributes.version", ascending=False).drop_duplicates(subset="source_table")

    if len(doi_ds) == 1:
        return doi_ds.iloc[0]["id"]
    else:
        return "DOI TBC"

In [7]:
df

NameError: name 'df' is not defined

In [ ]:
ds_doi("ALSPAC", "serology1m")

'DOI TBC'

In [ ]:
doi_ds = dcf.get_doi_datasets()[dcf.get_doi_datasets()["state"] == "findable"]
doi_ds["source_table"] = doi_ds["attributes.titles"].apply(lambda x: x[1]["title"] if len(x) > 1 else "NA")

doi_ds = doi_ds[doi_ds["source_table"] == "BCS70" + "_" + "bcs10_housing"]

if len(doi_ds) == 1:
    return doi_ds.iloc[0]["id"]
else:
    return "DOI TBC"

SyntaxError: 'return' outside function (1914446212.py, line 7)

In [ ]:
doi_ds = dcf.get_doi_datasets()[dcf.get_doi_datasets()["state"] == "findable"]
doi_ds["source_table"] = doi_ds["attributes.titles"].apply(lambda x: x[1]["title"] if len(x) > 1 else "NA")

doi_ds = doi_ds[doi_ds["source_table"] == "BCS70" + "_" + "bcs10_housing"]


In [ ]:
doi_ds

,id,attributes.titles,attributes.version,state,attributes.identifiers,creators,title,source_table
1,10.83126/ukllc-dataset-00032-01,"[{'lang': None, 'title': '1970 British Cohort ...",1,findable,"[{'identifier': 'BCS70', 'identifierType': 'UK...","University College London, UCL Social Research...",1970 British Cohort Study (BCS70): Age 46 Housing,BCS70_bcs10_housing


In [ ]:
dsvs = md.get_md_api_dsvs()
dsvs = dsvs[(dsvs["source"] == "BCS70")
            & (dsvs["table"] == "bcs10_housing")]
dsvs["source_table"] = dsvs["source"] + "_" + dsvs["table"]

ds_dois = dcf.get_doi_datasets()
ds_dois = ds_dois[ds_dois["state"] == "findable"]
ds_dois["source_table"] = ds_dois["attributes.titles"].apply(lambda x: x[1]["title"])
ds_dois["attributes.version"] = ds_dois["attributes.version"].apply(lambda x: int(x))
ds_dois = ds_dois[ds_dois["source_table"] == "BCS70" + "_" + "bcs10_housing"]

dsvs = dsvs.merge(ds_dois, left_on=["source_table", "version_num"], right_on=["source_table", "attributes.version"])[["source_table", "version_num", "version_date", "num_participants", "num_columns", "num_rows", "id"]]


dsvs["version_num"] = dsvs["version_num"].apply(
    lambda x: "Version " + str(int(x.split("v")[1])))
dsvs["version_date"] = dsvs["version_date"].apply(
    lambda x: datetime.strftime(datetime.strptime(
        str(int(x)), "%Y%m%d"), "%d %b %Y"))
dsvs["num_columns"] = dsvs["num_columns"].apply(lambda x: int(x))
dsvs["num_participants"] = dsvs["num_participants"].apply(
    lambda x: int(x))

dsvs2 = dsvs[["version_num",
                "version_date",
                "num_columns",
                "num_participants",
                "id"]].rename(
                columns={
                    "version_num": "Version Number",
                    "version_date": "Version Date",
                    "num_columns": "Number of Variables",
                    "num_participants": "Number of Participants",
                    "act": "Change Log"}
                    ).set_index("Version Number")
dsvs2.T

ValueError: You are trying to merge on object and int64 columns for key 'version_num'. If you wish to proceed you should use pd.concat

In [10]:
dsvs = md.get_md_api_dsvs()
dsvs = dsvs[(dsvs["source"] == "ALSPAC")
            & (dsvs["table"] == "serology1m")]
dsvs["source_table"] = dsvs["source"] + "_" + dsvs["table"]
dsvs["version_num"] = dsvs["version_num"].\
    apply(lambda x: int(x.replace("v", "")))

ds_dois = dcf.get_doi_datasets()
ds_dois = ds_dois[ds_dois["state"] == "findable"]
ds_dois["source_table"] = ds_dois["attributes.titles"].apply(lambda x: x[1]["title"])
ds_dois["attributes.version"] = ds_dois["attributes.version"].apply(lambda x: int(x))
ds_dois = ds_dois[ds_dois["source_table"] == "BCS70" + "_" + "bcs10_housing"]

dsvs2 = dsvs.merge(ds_dois, how="left", left_on=["source_table", "version_num"], right_on=["source_table", "attributes.version"])[["source_table", "version_num", "version_date", "num_participants", "num_columns", "num_rows", "id"]]

dsvs2["id"] = dsvs2["id"].fillna("TBC")
dsvs2["act"] = dsvs2["id"].apply(lambda x: "TBC" if x == "TBC" else x + "/activites")
dsvs2["num_participants"] = dsvs2["num_participants"].apply(lambda x: "N/A" if np.isnan(x) else int(x))
dsvs2["num_columns"] = dsvs2["num_columns"].apply(lambda x: int(x))
dsvs2["num_rows"] = dsvs2["num_rows"].apply(lambda x: int(x))

dsvs2 = dsvs2[["version_num",
                "version_date",
                "num_columns",
                "num_participants",
                "id",
                "act"]].rename(
                columns={
                    "version_num": "Version Number",
                    "version_date": "Version Date",
                    "num_columns": "Number of Variables",
                    "num_participants": "Number of Participants",
                    "id": "DOI",
                    "act": "Change Log"}
                    ).set_index("Version Number")
dsvs2.T

Version Number,1
Version Date,20220531.0
Number of Variables,20
Number of Participants,679
DOI,TBC
Change Log,TBC


In [9]:
dsvs = md.get_md_api_dsvs()
dsvs = dsvs[(dsvs["source"] == "ALSPAC")
            & (dsvs["table"] == "serology1m")]
dsvs["source_table"] = dsvs["source"] + "_" + dsvs["table"]
dsvs["version_num"] = dsvs["version_num"].\
    apply(lambda x: int(x.replace("v", "")))

ds_dois = dcf.get_doi_datasets()
ds_dois = ds_dois[ds_dois["state"] == "findable"]
ds_dois["source_table"] = ds_dois["attributes.titles"].apply(lambda x: x[1]["title"])
ds_dois["attributes.version"] = ds_dois["attributes.version"].apply(lambda x: int(x))
ds_dois = ds_dois[ds_dois["source_table"] == "BCS70" + "_" + "bcs10_housing"]

dsvs2 = dsvs.merge(ds_dois, how="left", left_on=["source_table", "version_num"], right_on=["source_table", "attributes.version"])[["source_table", "version_num", "version_date", "num_participants", "num_columns", "num_rows", "id"]]

dsvs2["id"] = dsvs2["id"].fillna("TBC")
dsvs2["act"] = dsvs2["id"].apply(lambda x: "TBC" if x == "TBC" else x + "/activites")
dsvs2

,source_table,version_num,version_date,num_participants,num_columns,num_rows,id,act
0,ALSPAC_serology1m,1,20220531.0,679.0,20.0,4978.0,TBC,TBC
